<a href="https://colab.research.google.com/github/QiuYue24/CASA0025_1/blob/main/W04_postgis2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Spatial Joins Exercises

Here\'s a reminder of some of the functions we have seen. Hint: they
should be useful for the exercises!

-   `sum(expression)`: aggregate to
    return a sum for a set of records
-   `count(expression)`: aggregate to
    return the size of a set of records
-   `ST_Area(geometry)` returns the
    area of the polygons
-   `ST_AsText(geometry)` returns WKT `text`
-   `ST_Contains(geometry A, geometry B)` returns the true if geometry A contains geometry B
-   `ST_Distance(geometry A, geometry B)` returns the minimum distance between geometry A and
    geometry B
-   `ST_DWithin(geometry A, geometry B, radius)` returns the true if geometry A is radius distance or less from geometry B
-   `ST_GeomFromText(text)` returns `geometry`
-   `ST_Intersects(geometry A, geometry B)` returns the true if geometry A intersects geometry B
-   `ST_Length(linestring)` returns the length of the linestring
-   `ST_Touches(geometry A, geometry B)` returns the true if the boundary of geometry A touches geometry B
-   `ST_Within(geometry A, geometry B)` returns the true if geometry A is within geometry B


Uncomment and run the following cell to install the required packages.


In [1]:
%pip install duckdb leafmap lonboard
import duckdb
import leafmap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.2/515.2 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 884.2/884.2 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.7/213.7 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 59.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 68.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 74.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 59.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 765.5/765.5 kB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.2/194.2 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.4/72.4

Download the [nyc_data.zip](https://github.com/opengeos/data/raw/main/duckdb/nyc_data.zip) dataset using leafmap. The zip file contains the following datasets. Create a new DuckDB database and import the datasets into the database. Each dataset should be imported into a separate table.

- nyc_census_blocks
- nyc_homicides
- nyc_neighborhoods
- nyc_streets
- nyc_subway_stations

In [2]:
url = "https://open.gishub.org/data/duckdb/nyc_data.db.zip"
leafmap.download_file(url, unzip=True)

Downloading...
From: https://open.gishub.org/data/duckdb/nyc_data.db.zip
To: /content/nyc_data.db.zip
100%|██████████| 8.60M/8.60M [00:00<00:00, 82.9MB/s]


Extracting files...


'/content/nyc_data.db.zip'

In [3]:
con = duckdb.connect("nyc_data.db")

In [4]:
con.install_extension("spatial")
con.load_extension("spatial")

In [5]:
con.sql("SHOW TABLES;")

┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ nyc_census_blocks   │
│ nyc_homicides       │
│ nyc_neighborhoods   │
│ nyc_streets         │
│ nyc_subway_stations │
└─────────────────────┘

In [ ]:
con.sql("SELECT * FROM nyc_subway_stations LIMIT 5").df()  # 查看前5行

,OBJECTID,ID,NAME,ALT_NAME,CROSS_ST,LONG_NAME,LABEL,BOROUGH,NGHBHD,ROUTES,TRANSFERS,COLOR,EXPRESS,CLOSED,geom
0,1.0,376.0,Cortlandt St,None,Church St,"Cortlandt St (R,W) Manhattan","Cortlandt St (R,W)",Manhattan,None,"R,W","R,W",YELLOW,None,None,"[0, 0, 24, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,..."
1,2.0,2.0,Rector St,None,None,Rector St (1) Manhattan,Rector St (1),Manhattan,None,1,1,RED,None,None,"[0, 0, 24, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,..."
2,3.0,1.0,South Ferry,None,None,South Ferry (1) Manhattan,South Ferry (1),Manhattan,None,1,1,RED,None,None,"[0, 0, 24, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,..."
3,4.0,125.0,138th St,Grand Concourse,Grand Concourse,"138th St / Grand Concourse (4,5) Bronx","138th St / Grand Concourse (4,5)",Bronx,None,"4,5","4,5",GREEN,None,None,"[0, 0, 24, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,..."
4,5.0,126.0,149th St,Grand Concourse,Grand Concourse,149th St / Grand Concourse (4) Bronx,149th St / Grand Concourse (4),Bronx,None,4,"2,4,5",GREEN,express,None,"[0, 0, 24, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,..."


In [ ]:
con.sql("SELECT * FROM nyc_census_blocks LIMIT 5").df()  # 查看前5行

,BLKID,POPN_TOTAL,POPN_WHITE,POPN_BLACK,POPN_NATIV,POPN_ASIAN,POPN_OTHER,BORONAME,geom
0,360850009001000,97,51,32,1,5,8,Staten Island,"[5, 4, 184, 0, 0, 0, 0, 0, 55, 3, 13, 73, 151,..."
1,360850020011000,66,52,2,0,7,5,Staten Island,"[5, 4, 136, 0, 0, 0, 0, 0, 178, 58, 13, 73, 72..."
2,360850040001000,62,14,18,2,25,3,Staten Island,"[5, 4, 120, 0, 0, 0, 0, 0, 82, 227, 12, 73, 55..."
3,360850074001000,137,92,12,0,13,20,Staten Island,"[5, 4, 184, 0, 0, 0, 0, 0, 204, 85, 13, 73, 10..."
4,360850096011000,289,230,0,0,32,27,Staten Island,"[5, 4, 89, 0, 0, 0, 0, 0, 107, 247, 12, 73, 7,..."


In [ ]:
con.sql("SELECT * FROM nyc_homicides LIMIT 5").df()  # 查看前5行

NameError: name 'con' is not defined

In [ ]:
con.sql("SELECT * FROM nyc_neighborhoods LIMIT 5").df()  # 查看前5行

,BORONAME,NAME,geom
0,Brooklyn,Bensonhurst,"[5, 4, 41, 0, 0, 0, 0, 0, 54, 71, 14, 73, 198,..."
1,Manhattan,East Village,"[5, 4, 152, 0, 0, 0, 0, 0, 35, 215, 14, 73, 13..."
2,Manhattan,West Village,"[5, 4, 91, 0, 0, 0, 0, 0, 161, 95, 14, 73, 212..."
3,The Bronx,Throggs Neck,"[5, 4, 141, 0, 0, 0, 0, 0, 128, 232, 17, 73, 1..."
4,The Bronx,Wakefield-Williamsbridge,"[5, 4, 126, 0, 0, 0, 0, 0, 83, 85, 17, 73, 17,..."


In [ ]:
con.sql("SELECT * FROM nyc_streets LIMIT 5").df()  # 查看前5行

,ID,NAME,ONEWAY,TYPE,geom
0,1,Shore Pky S,None,residential,"[4, 4, 112, 0, 0, 0, 0, 0, 23, 66, 15, 73, 202..."
1,2,None,None,footway,"[4, 4, 240, 0, 0, 0, 0, 0, 80, 57, 15, 73, 35,..."
2,3,Avenue O,None,residential,"[4, 4, 225, 0, 0, 0, 0, 0, 228, 63, 15, 73, 21..."
3,4,Walsh Ct,None,residential,"[4, 4, 48, 0, 0, 0, 0, 0, 139, 62, 15, 73, 102..."
4,5,None,None,motorway_link,"[4, 4, 48, 0, 0, 0, 0, 0, 176, 53, 15, 73, 16,..."


1. **What subway station is in \'Little Italy\'? What subway route is it on?**

In [ ]:
con.sql("SELECT * FROM nyc_neighborhoods WHERE LOWER(name) LIKE '%little italy%'").df()

,BORONAME,NAME,geom
0,Manhattan,Little Italy,"[5, 4, 136, 0, 0, 0, 0, 0, 221, 175, 14, 73, 2..."


In [ ]:
con.sql("""

SELECT s.name AS sub_station_name, s.routes
FROM nyc_subway_stations s
JOIN nyc_neighborhoods n ON ST_Contains(n.geom, s.geom) #检查某个点是否落在某个多边形区域内，因为地铁站是点，行政区是多边形
WHERE n.name = 'Little Italy';

""")

┌──────────────────┬─────────┐
│ sub_station_name │ ROUTES  │
│     varchar      │ varchar │
├──────────────────┼─────────┤
│ Spring St        │ 6       │
└──────────────────┴─────────┘

2. **What are all the neighborhoods served by the 6-train?** (Hint: The `routes` column in the `nyc_subway_stations` table has values like \'B,D,6,V\' and \'C,6\')


In [9]:
con.sql("""

SELECT DISTINCT n.name, n.boroname
FROM nyc_neighborhoods n, nyc_subway_stations s
WHERE ST_Contains(n.geom, s.geom) AND s.routes LIKE '%6%';

""")

┌────────────────────┬───────────┐
│        NAME        │ BORONAME  │
│      varchar       │  varchar  │
├────────────────────┼───────────┤
│ Financial District │ Manhattan │
│ Little Italy       │ Manhattan │
│ Upper East Side    │ Manhattan │
│ East Harlem        │ Manhattan │
│ Mott Haven         │ The Bronx │
│ Hunts Point        │ The Bronx │
│ South Bronx        │ The Bronx │
│ Chinatown          │ Manhattan │
│ Greenwich Village  │ Manhattan │
│ Gramercy           │ Manhattan │
│ Murray Hill        │ Manhattan │
│ Midtown            │ Manhattan │
│ Yorkville          │ Manhattan │
│ Soundview          │ The Bronx │
│ Parkchester        │ The Bronx │
├────────────────────┴───────────┤
│ 15 rows              2 columns │
└────────────────────────────────┘

3. **After 9/11, the \'Battery Park\' neighborhood was off limits for several days. How many people had to be evacuated?**

In [ ]:
con.sql("""

SELECT sum(blk.popn_total) AS total_evacuated_people
FROM nyc_census_blocks blk
JOIN nyc_neighborhoods n ON ST_Intersects(blk.geom, n.geom)
WHERE n.name = 'Battery Park'


""")

┌────────────────────────┐
│ total_evacuated_people │
│         int128         │
├────────────────────────┤
│                  17153 │
└────────────────────────┘

In [ ]:
con.sql("""

SELECT blk.BLKID, SUM(blk.popn_total) AS total_evacuated_people
FROM nyc_census_blocks blk
JOIN nyc_neighborhoods n ON ST_Intersects(blk.geom, n.geom)
WHERE n.name = 'Battery Park'
GROUP BY blk.BLKID
ORDER BY total_evacuated_people DESC;

""")


┌─────────────────┬────────────────────────┐
│      BLKID      │ total_evacuated_people │
│     varchar     │         int128         │
├─────────────────┼────────────────────────┤
│ 360610317031001 │                   2926 │
│ 360610317032005 │                   2355 │
│ 360610013002005 │                   1453 │
│ 360610317044001 │                   1416 │
│ 360610317041000 │                   1091 │
│ 360610021002003 │                    991 │
│ 360610317032000 │                    911 │
│ 360610039003000 │                    851 │
│ 360610317042000 │                    698 │
│ 360610317044000 │                    695 │
│        ·        │                      · │
│        ·        │                      · │
│        ·        │                      · │
│ 360610317044003 │                      0 │
│ 360610317042002 │                      0 │
│ 360610021002008 │                      0 │
│ 360610317032006 │                      0 │
│ 360610317044004 │                      0 │
│ 36061031

4. **What neighborhood has the highest population density (persons/km2)?**


In [ ]:
#需要同时按 n.name 和 n.geom 进行分组(GROUP BY)，确保每个社区计算一个密度值。

In [6]:
con.sql("""

SELECT sum(blk.popn_total) / (ST_Area(n.geom) / 1e6) AS pop_density
FROM nyc_census_blocks blk
JOIN nyc_neighborhoods n ON ST_Intersects(blk.geom, n.geom)
GROUP BY n.name, n.geom
ORDER BY pop_density DESC
LIMIT 1;

""")

┌───────────────────┐
│    pop_density    │
│      double       │
├───────────────────┤
│ 68435.13283772678 │
└───────────────────┘

When you're finished, you can check your answers [here](https://postgis.net/workshops/postgis-intro/joins_exercises.html).

# Ship-to-Ship Transfer Detection

Now for a less structured exercise. We're going to look at ship-to-ship transfers. The idea is that two ships meet up in the middle of the ocean, and one ship transfers cargo to the other. This is a common way to avoid sanctions, and is often used to transfer oil from sanctioned countries to other countries. We're going to look at a few different ways to detect these transfers using AIS data.

In [7]:
%pip install duckdb duckdb-engine jupysql

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.8/192.8 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.7 MB/s eta 0:00:00


In [10]:
import duckdb
import pandas as pd

# Import jupysql Jupyter extension to create SQL cells
%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
%sql duckdb:///:memory:

In [11]:
%%sql
INSTALL httpfs;
LOAD httpfs;
INSTALL spatial;
LOAD spatial;

,Success


## Step 1

Create a spatial database using the following AIS data:

https://storage.googleapis.com/qm2/casa0025_ships.csv

Each row in this dataset is an AIS 'ping' indicating the position of a ship at a particular date/time, alongside vessel-level characteristics.

It contains the following columns:
* `vesselid`: A unique numerical identifier for each ship, like a license plate
* `vessel_name`: The ship's name
* `vsl_descr`: The ship's type
* `dwt`: The ship's Deadweight Tonnage (how many tons it can carry)
* `v_length`: The ship's length in meters
* `draught`: How many meters deep the ship is draughting (how low it sits in the water). Effectively indicates how much cargo the ship is carrying
* `sog`: Speed over Ground (in knots)
* `date`: A timestamp for the AIS signal
* `lat`: The latitude of the AIS signal (EPSG:4326)
* `lon`: The longitude of the AIS signal (EPSG:4326)

Create a table called 'ais' where each row is a different AIS ping, with no superfluous information. Construct a geometry column.

Create a second table called 'vinfo' which contains vessel-level information with no superfluous information.

You can set a spatial index on each of these tables as follows:

`CREATE INDEX index_name ON table_name USING RTREE(geom);`

In [13]:
import pandas as pd

# 下载数据
url = "https://storage.googleapis.com/qm2/casa0025_ships.csv"
df = pd.read_csv(url)
# 显示前几行数据
df.head()

,vesselid,vessel_name,vsl_descr,dwt,v_length,draught,sog,date,lat,lon,geom
0,350053,30 Let Pobedy,general cargo,5150.0,NaN,3.5,5.2,2022-07-25 02:53:29,45.151777,36.513327,POINT (36.5133266666667 45.1517766666667)
1,350053,30 Let Pobedy,general cargo,5150.0,NaN,3.5,0.7,2022-07-25 03:09:37,45.146487,36.520780,POINT (36.52078 45.1464866666667)
2,350053,30 Let Pobedy,general cargo,5150.0,NaN,3.5,0.7,2022-07-25 03:13:58,45.146218,36.521965,POINT (36.521965 45.1462183333333)
3,350053,30 Let Pobedy,general cargo,5150.0,NaN,3.5,0.1,2022-07-25 04:16:06,45.145058,36.522020,POINT (36.52202 45.1450583333333)
4,350053,30 Let Pobedy,general cargo,5150.0,NaN,3.5,0.0,2022-07-25 05:20:17,45.144933,36.521848,POINT (36.5218483333333 45.1449333333333)


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101328 entries, 0 to 101327
Data columns (total 11 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   vesselid     101328 non-null  int64  
 1   vessel_name  101328 non-null  object 
 2   vsl_descr    101328 non-null  object 
 3   dwt          101328 non-null  float64
 4   v_length     99631 non-null   float64
 5   draught      101308 non-null  float64
 6   sog          101090 non-null  float64
 7   date         101328 non-null  object 
 8   lat          101328 non-null  float64
 9   lon          101328 non-null  float64
 10  geom         101328 non-null  object 
dtypes: float64(6), int64(1), object(4)
memory usage: 8.5+ MB


In [15]:
import duckdb

# 会创建一个名为 ais_data.db 的 DuckDB 数据库文件（如果文件不存在），存在则是连接数据库
con = duckdb.connect("ais_data.db")

In [16]:
# 安装并加载空间扩展
con.install_extension("spatial")
con.load_extension("spatial")

In [18]:
# 创建 AIS 表（包含几何列）
con.sql("""
CREATE TABLE ais (
    vesselid INTEGER,           -- 船只唯一ID
    vessel_name TEXT,           -- 船只名称
    vsl_descr TEXT,             -- 船只类型描述
    dwt INTEGER,                -- 载重吨位 (Deadweight Tonnage)
    v_length FLOAT,             -- 船只长度（米）
    draught FLOAT,              -- 吃水深度（米）
    sog FLOAT,                  -- 航速（节）
    date TIMESTAMP,             -- AIS 发送时间
    lat DOUBLE,                 -- 纬度（EPSG:4326）
    lon DOUBLE,                 -- 经度（EPSG:4326）
    geometry GEOMETRY           -- 空间列，存储坐标点
);
""")


In [ ]:
con.sql("""

CREATE or REPLACE TABLE samples (name VARCHAR, geom GEOMETRY);

SELECT * FROM samples;

  """)

In [19]:
# 直接导入 Pandas DataFrame 到 DuckDB
con.register("ais_temp", df)

con.sql("""
INSERT INTO ais (vesselid, vessel_name, vsl_descr, dwt, v_length, draught, sog, date, lat, lon)
SELECT vesselid, vessel_name, vsl_descr, dwt, v_length, draught, sog, date, lat, lon FROM ais_temp;
""")

In [27]:
#由于 DuckDB 不支持自动生成 geometry 列，所以手动设置，默认叫 geometry 不能直接改成 geom
con.sql("""
UPDATE ais
SET geometry = ST_Point(lon, lat);
""")

In [22]:
#检查表结构
con.sql("PRAGMA table_info(ais);").df()

,cid,name,type,notnull,dflt_value,pk
0,0,vesselid,INTEGER,False,None,False
1,1,vessel_name,VARCHAR,False,None,False
2,2,vsl_descr,VARCHAR,False,None,False
3,3,dwt,INTEGER,False,None,False
4,4,v_length,FLOAT,False,None,False
5,5,draught,FLOAT,False,None,False
6,6,sog,FLOAT,False,None,False
7,7,date,TIMESTAMP,False,None,False
8,8,lat,DOUBLE,False,None,False
9,9,lon,DOUBLE,False,None,False


In [23]:
#预览数据
con.sql("SELECT * FROM ais LIMIT 5;").df()

,vesselid,vessel_name,vsl_descr,dwt,v_length,draught,sog,date,lat,lon,geometry
0,350053,30 Let Pobedy,general cargo,5150,NaN,3.5,5.2,2022-07-25 02:53:29,45.151777,36.513327,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
1,350053,30 Let Pobedy,general cargo,5150,NaN,3.5,0.7,2022-07-25 03:09:37,45.146487,36.520780,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
2,350053,30 Let Pobedy,general cargo,5150,NaN,3.5,0.7,2022-07-25 03:13:58,45.146218,36.521965,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
3,350053,30 Let Pobedy,general cargo,5150,NaN,3.5,0.1,2022-07-25 04:16:06,45.145058,36.522020,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
4,350053,30 Let Pobedy,general cargo,5150,NaN,3.5,0.0,2022-07-25 05:20:17,45.144933,36.521848,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."


In [28]:
#DuckDB 不支持 ALTER TABLE MODIFY COLUMN，所以我们需要先添加新列
con.sql("""
ALTER TABLE ais ADD COLUMN geom GEOMETRY;
""")

In [30]:
#再更新 geom 列
con.sql("""
UPDATE ais
SET geom = ST_Point(lon, lat);
""")

## Step 2

Use a spatial join to identify ship-to-ship transfers in this dataset.
Two ships are considered to be conducting a ship to ship transfer IF:

* They are within 500 meters of each other
* For more than two hours
* And their speed is lower than 1 knot

Some things to consider: make sure you're not joining ships with themselves. Try working with subsets of the data first while you try different things out.

In [31]:
con.sql("""

SELECT
  a1.vesselid AS ship1,
  a2.vesselid AS ship2,
  a1.date AS start,
  a2.date AS end
FROM ais AS a1
JOIN ais AS a2
ON ST_DWithin(a1.geom, a2.geom, 500) -- 500米内
AND ABS(EXTRACT(EPOCH FROM (a2.date - a1.date))) > 7200 -- 2小时以上
AND a1.vesselid <> a2.vesselid;

""")

┌────────┬────────┬─────────────────────┬─────────────────────┐
│ ship1  │ ship2  │        start        │         end         │
│ int32  │ int32  │      timestamp      │      timestamp      │
├────────┼────────┼─────────────────────┼─────────────────────┤
│ 323648 │ 350053 │ 2022-06-28 14:31:37 │ 2022-07-25 02:53:29 │
│ 323648 │ 350053 │ 2022-06-28 14:41:56 │ 2022-07-25 02:53:29 │
│ 323648 │ 350053 │ 2022-06-28 14:51:59 │ 2022-07-25 02:53:29 │
│ 323648 │ 350053 │ 2022-06-28 15:02:17 │ 2022-07-25 02:53:29 │
│ 323648 │ 350053 │ 2022-06-28 15:24:19 │ 2022-07-25 02:53:29 │
│ 323648 │ 350053 │ 2022-06-28 16:12:20 │ 2022-07-25 02:53:29 │
│ 323648 │ 350053 │ 2022-06-28 18:06:21 │ 2022-07-25 02:53:29 │
│ 323648 │ 350053 │ 2022-06-28 19:06:25 │ 2022-07-25 02:53:29 │
│ 323648 │ 350053 │ 2022-06-28 20:12:20 │ 2022-07-25 02:53:29 │
│ 323648 │ 350053 │ 2022-06-28 21:12:22 │ 2022-07-25 02:53:29 │
│    ·   │    ·   │          ·          │          ·          │
│    ·   │    ·   │          ·          